In [1]:
# 사용하려는 라이브러리 호출 및 저장
import bok_da as bd
import numpy as np
import pandas as pd
import scipy.stats as sp
import matplotlib.pyplot as plt
import seaborn as sns

#### 본 매뉴얼은 단위근 검정에 대해 다룬다.
- 작성자: 고려대학교 경제학과 강규호 교수, 디지털신기술팀 이창훈 과장

## 1. 구성함수 (상위함수와 사용되는 하위함수)
- `longrun_var`: 장기 분산 추정 (Bartlett 또는 Quadratic Kernel) 사용

- `mic`: 수정된 정보기준 (Modified Information Criteria)를 활용한 최적 시차 추정

- `UR_ADF_GLS`: GLS 추세 제거법 (GLS detrending)을 사용한 ADF 계열의 단위근 검정

## 2. 데이터 불러오기

데이터는 한국의 전기대비 실질 GDP 성장률, CD91일 금리, 인플레이션율 소비자 가격 지수 (Consumer Price Index, CPI), CD91일 금리로 1999년 4분기부터 2023년 4분끼지의 분기별 자료이다. 해당 데이터는 ECOS (https://ecos.bok.or.kr/#/)에서 다운로드 받았으며 컴퓨터에 "python_data.xlsx"라는 이름의 엑셀파일의 "KORmacro"라는 시트에 저장되어 있다. 이때, 인플레이션율의 경우 소비자 가격 지수 (Consumer Price Index, CPI)의 전년동기대비 치분 (YoY)을 통해 미리 변환한 것이다.

`data` 폴더에서 `python_data.xlsx`를 불러온 후 파이썬 표준 자료형인 `pandas.DataFrame`으로 전처리를 수행한다. `DataFrame`은 테이블 형태의 데이터의 행(날짜)과 열(변수명) 정보를 가지고 있어, 행렬이나 벡터형태로 분석하는 것보다 유용하다. `df`라는 이름으로 할당하는 것이 일반적이다. 데이터프레임에 익숙하지 않은 경우 아래 셀의 코드를 줄별로 여러 셀로 나누어 실행해보는 것을 추천한다. 원천기관에서 raw 데이터를 불러오고 전처리하는 과정이 복잡해 보일 수 있지만, 한번 해두면 크게 코드를 수정할 필요없이 새로운 데이터에 대해서도 분석 가능한 형태로 전처리 할 수 있기 때문에 장기적으로 효율적인 방법이다.

In [2]:
df = pd.read_excel('data/python_data.xlsx', sheet_name='KORmacro') # 데이터 엑셀파일 불러오기
df = df.set_index('date') # date 명의 열을 인덱스로 지정한다.
df.index.name = None # 인덱스 이름 date를 제거한다.
df.index = df.index.str.replace('/', '-') # 2000/03 형식을 pandas가 날짜로 인식하도록 2000-3형식으로 변환한다.
df.index = pd.to_datetime(df.index) + pd.offsets.QuarterEnd(0) # 인덱스(행)를 날짜로 인식하도록 변환한다.
df.index.freq = pd.offsets.QuarterEnd() # 인덱스의 frequency를 분기말(2000-03-31)로 설정한다.
column_order = ['inf', 'rgdp', 'cd91'] # df의 열 순서를 리스트로 미리 할당
df = df[column_order] # df의 열 순서를 column_order에 맞게 변경
df

,inf,rgdp,cd91
2000-12-31,2.508257,-0.3,6.97
2001-03-31,3.627876,1.2,5.95
2001-06-30,4.919778,1.3,5.78
2001-09-30,4.158456,1.4,4.97
2001-12-31,3.244327,1.8,4.58
...,...,...,...
2022-12-31,5.083054,-0.3,3.91
2023-03-31,4.496285,0.3,3.64
2023-06-30,3.212650,0.6,3.63
2023-09-30,3.079117,0.6,3.74


## 3. 장기 분산 추정
`bok_da` 라이브러리의 `longrun_var`라는 함수를 사용하면 시계열의 장기 분산을 추정할 수 있다. 이때, `longrun_var`라는 함수는 다음과 구성되어 있다.

> *<p style="font-size:15px">`jhat = longrun_var(y, window=None)`</p>*
> *<p style="font-size:15px">장기 분산 추정</p>*
> - **입력**:
</br>
    y: 추정하고자하는 변수, 단변수 또는 다변수 모두 가능
    </br>
    window: 사용하고자하는 윈도우의 종류
    </br>
        'bartlett': Bartlett Window (Newey-West 추정량)
        </br>
        'quadratic': AR(1) 과정을 통해 근사한 밴드 넓이 파라미터 (bandwidth parameter)를 사용한 Quadratic Window (default)
        </br></br>
> - **출력**:
</br>
    jhat: 장기 분산 추정치
    </br>

### (참고: 장기 분산 추정)
다음과 같은 모형을 고려해보자.
$\begin{equation}
    y_{t} = x_{t}^{\prime}\beta + u_{t} \notag
\end{equation}$

OLS 기법을 사용한 $\beta$라는 계수값은 다음과 같은 점근적 분포를 따른다.
\begin{align}
    \sqrt{T}(\hat{\beta}-\beta) & = \underbrace{\Bigg(\frac{1}{T}\sum_{t=1}^{T}{x_{t}x_{t}^{\prime}}\Bigg)^{-1}}_{\to^{p}B_{\infty}^{-1}}\cdot\underbrace{\frac{1}{\sqrt{T}}\sum_{t=1}^{T}{x_{t}u_{t}}}_{\to^{d}\mathcal{N}(0,J_{\infty})} \notag \\
    & \to^{d} B_{\infty}^{-1}\cdot\mathcal{N}(0,J_{\infty}) \notag \\
    & \approx^{d}\mathcal{N}(0,B_{\infty}^{-1}\cdot J_{\infty}\cdot {B_{\infty}^{-1}}^{\prime}) \notag
\end{align}
이때, 점근적 분포의 분산항을 이분산과 자기공분산에 견고한 (Heteroskedasticity and Autocorrelation Consistent, HAC) 분산-공분산 행렬이라고 부른다.

한편, HAC 분산-공분산 행렬의 추정에 있어서 $J_{\infty}=\sum_{j=-\infty}^{\infty}{\Gamma(j)}$를 추정하는 것이 상당히 중요하며, 이는 스펙트럼 분석 (spectral analysis)로 할 수 있다. 정의에 따라, 다변수 스펙트럼 밀도 함수는 다음과 같다.
$\begin{equation}
    h(w) = \frac{1}{2\pi}\sum_{\tau=-\infty}^{\infty}{\exp{-i\cdot w\cdot \tau}R(\tau)} \notag
\end{equation}$
이후,
\begin{equation}
    \exp{(-i\cdot w\cdot \tau)} = \cos{(ws)}\pm\sin{(ws)} \notag
\end{equation}
와
\begin{equation}
    \sin{(-x)} = \sin{(x)} \notag
\end{equation}
라는 관계를 사용하면, 위 스펙트럴 밀도함수는 다음과 같이 변환하여 표현할 수 있다.
\begin{equation}
    h(w) = \frac{1}{2\pi}\sum_{\tau=-\infty}^{\infty}{\cos{(w\tau)}R(\tau)} \tag{1}
\end{equation}

더 나아가, $R(\tau)$를 표본 추정량으로 대체하고, $-\infty$에서 $\infty$까지의 무한합을 $-(T-1)$부터 $(T-1)$까지의 유한합으로 대체하면, 식 (1)의 스펙트럴 밀도함수는 다음과 같은 형태로 대체될 수 있다.
\begin{equation}
    \hat{h}_{**}(w) = \frac{1}{2\pi}\sum_{\tau=-(T-1)}^{T-1}{k\Bigg(\frac{\tau}{s_{T}}\Bigg)\cos{(w\tau)}\hat{R}(\tau)} \notag
\end{equation}
이때, 스펙트럴 밀도함수는 $k(\cdot)$로 표기된 커넬 함수 (kernel function)와 $s_{T}$라는 밴드 넓이 파라미터 (band width parameter)에 따라 달라진다는 특징을 가진다.

- 커넬 함수

    우선, 다양한 커넬 함수의 후보 중에서 Bartlett 커넬과 Quadratic 커넬이 가장 널리 사용된다.

    1. Bartlett 커넬

        $\begin{equation}
        k_{BT}(x) = \begin{cases}
            1-|x| & \text{ for }|x|\leq 1 \\
            0 & \text{ otherwise}
        \end{cases} \notag
    \end{equation}$

    2. Qudratic 커넬

        $\begin{equation}
            k_{QS}(x) = \frac{25}{12\pi^{2}x^{2}}\Bigg(\frac{\sin{(\frac{6\pi x}{5})}}{\frac{6\pi x}{5}}-\cos{\Bigg(\frac{6\pi x}{5}\Bigg)}\Bigg) \notag
        \end{equation}$

- 밴드 넓이 파라미터

    한편, Andrews (1991)은 다음과 같이 최적 추정량을 통해 각 커넬 함수에 대한 밴드 넓이 파라미터를 제시하였다. 위에서 제시된 커넬 함수에 대한 밴드 넓이 파라미터는 다음과 같은 추정량을 사용한다.

    1. Bartlett 커넬: $\hat{s}_{T}=1.1447(\hat{\alpha}(1)\cdot T)^{1/3}$
    2. Qudratic 커넬: $\hat{s}_{T}=1.3221(\hat{\alpha}(2)\cdot T)^{1/5}$

    이때, $\hat{\alpha}$는 다음과 같이 AR(1) 모형을 통해 근사하여 도출한다.
    $\begin{equation}
        y_{t} = \rho_{a}y_{t-1} + u_{t}\text{, }u_{t}\sim WN(0,\sigma_{a}^{2}) \notag
    \end{equation}$
    라는 AR(1) 과정은 다음과 같은 계수 및 분산-공분산 추정량을 가진다.
    $\begin{align}
        & \hat{\rho}_{a} = \Bigg(\sum\hat{V}_{at-1}^{2}\Bigg)^{-1}\Bigg(\sum{\hat{V}_{at-1}\hat{V}_{at}}\Bigg) \notag \\
        & \hat{\sigma}_{a} = \frac{1}{T}\sum{\Bigg(\hat{V}_{at}-\hat{\rho}_{a}\hat{V}_{at-1}\Bigg)^{2}} \notag
    \end{align}$
    이를 사용하면, 다음과 같은 수식을 통해 $\hat{\alpha}(1)$과 $\hat{\alpha}(2)$를 도출할 수 있다.
    $\begin{align}
        & \hat{\alpha}(1) = \Bigg(\sum_{a=1}^{p}{w_{a}\frac{4\hat{\rho}_{a}^{2}\hat{\sigma}_{a}^{4}}{(1-\hat{\rho}_{a})^{6}(1+\hat{\rho}_{a})^{2}}}\Bigg) / \Bigg(\sum_{a=1}^{p}{w_{a}\frac{\hat{\sigma}_{a}^{4}}{(1-\hat{\rho}_{a})^{4}}}\Bigg) \notag \\
        & \hat{\alpha}(2) = \Bigg(\sum_{a=1}^{p}{w_{a}\frac{4\hat{\rho}_{a}^{2}\hat{\sigma}_{a}^{4}}{(1-\hat{\rho}_{a})^{8}}}\Bigg) / \Bigg(\sum_{a=1}^{p}{w_{a}\frac{\hat{\sigma}_{a}^{4}}{(1-\hat{\rho}_{a})^{4}}}\Bigg) \notag
    \end{align}$

이때, Bartlett 커넬을 사용하여 추정한 분산-공분산 통계량을 Newey-West HAC 분산-공분산 추정량이라고 부른다. 다만, Andrews (1991)의 경우, 다른 커넬 함수를 사용하는 경우에 비해 수렴의 속도가 빠른 것과 같은 다양한 좋은 성질들에 근거하여 Quadratic 커넬을 사용하는 것을 추천한다.

### (예시: 인플레이션 장기 분산)
앞에서 생성한 df에서 인플레이션(inf)의 장기 분산을 두 가지 윈도우를 사용하여 추정해보자. 우선 `window`라는 입력값을 사용하지 않을 경우, default 옵션인 AR(1) 과정을 통해 근사한 밴드 넓이 파라미터를 사용한 Quadratic Window를 통한 장기분산 추정치를 얻을 수 있다.

In [3]:
from bok_da.ts.test import longrun_var

In [7]:
jhat = longrun_var(df.inf)
print(jhat)

[[16.77947745]]


해당 결과를 통해 한국의 전년동기대비 인플레이션이라는 변수는 약 16.78의 분산 추정치를 가진다는 것을 알 수 있다.

한편, 하나의 변수가 아니라 다변수 사이의 장기 분산-공분산 행렬을 추정할수도 있다. 이 경우, 입력값에 하나의 변수가 아닌 여러개의 변수로 이루어진 행렬을 넣으면 된다. 예를 들어, `y`라는 변수로 저장한 한국의 전년동기대비 인플레이션, 전기대비 실질 GDP 성장률, CD91일 금리의 장기 분산-공분산 행렬은 다음과 같이 추정할 수 있다. 이때, `window`라는 입력값을 사용하지 않거나, `window='quadratic'`라는 입력값을 넣을 경우 이전과 같이 Quadratic Window를 사용하여 장기 분산-공분산 행렬을 추정하게 된다.

In [8]:
jhat = longrun_var(df, window='quadratic')
print(jhat)

[[16.87433884  4.39808238 24.91487207]
 [ 4.39808238  1.73581932  8.56886919]
 [24.91487207  8.56886919 44.17032799]]


추정 결과, 한국의 인플레이션, 실질 GDP 성장률, CD91일 금리는 각각 약 16.87, 1.74, 44.17의 장기 분산 추정치를 가지는 것을 알 수 있다. 또한, 인플레이션과 실질 GDP 성장률은 약 4.40, 인플레이션과 CD91일 금리는 약 24.91의 장기 공분산 추청치를 가지는 것으로 나타났다. 마지막으로 실질 GDP 성장률과 CD91일 금리의 경우 약 8.57의 장기 공분산을 가지는 것으로 추정되었다.

`window='bartlett'`이라는 옵션을 통해 Quadratic window가 아닌 Bartlett window를 사용한 장기분산 추정치를 구할 수 있다. 이때, 이전과 마찬가지로 입력하는 데이터의 형태에 따라 단변수인 경우와 다변수인 경우 모두를 추정할 수 있다. 이전과 동일하게 `y`라는 변수로 저장한 한국의 전년동기대비 인플레이션, 전기대비 실질 GDP 성장률, CD91일 금리의 장기 분산-공분산 행렬은 다음과 같이 추정할 수 있다.

In [9]:
jhat = longrun_var(df, window='bartlett')
print(jhat)

[[14.929749    2.92541954 18.7795003 ]
 [ 2.92541954  1.43478183  6.1541235 ]
 [18.7795003   6.1541235  32.57784249]]


이처럼 Bartlett window를 사용하여 추정한 장기분산 값은 Newey-West HAC (Heteroskedasticity and Autocorrelation Consistent) 추정량이라고 부르기도 한다. 추정 결과, Newey-West 통계량의 전체적인 장기분산 및 공분산은 Quadratic window를 사용한 경우보다 과소추정되는 것을 확인할 수 있다. 실제로 몬테카를로 시뮬레이션을 사용해보면 장기분산을 더 정확히 추정하는 것은 default 옵션인 AR(1) 과정을 통해 근사한 밴드 넓이 파라미터를 사용한 Quadratic Window 기법이라는 것을 알 수 있다.

## 4. 단위근 검정

`bok_da` 라이브러리의  `adf_test_gls_detrending`라는 함수를 사용하여 GLS 추세제거 (GLS detrending)를 한 ADF (Adjusted Dicky-Fuller) 단위근 검정을 실시할 수 있다. 이때, `adf_test_gls_detrending`라는 함수는 다음과 같이 구성되어 있다.

> *<p style="font-size:15px">`res = adf_test_gls_detrending(y, m, ic=None)`</p>*
> *<p style="font-size:15px">GLS 추세 제거를 한 ADF 단위근 검정</p>*
> - **입력**:
</br>
    y: 추정하고자하는 변수, 단변수만 가능
    </br>
    m: 사용하고자하는 모형의 종류
    </br>
          'constant': 상수항만 존재하는 모형
        </br>
          'trend': 상수항과 선형 추세항이 존재하는 모형
        </br>
    ic: 수정된 정보기준의 종류
    </br>
          'maic': MAIC, penalty = 2
        </br>
          'mbic': MBIC, penalty = $\ln{(T-pmax-1)}$ (default)
        </br>
> - **출력**:
</br>
    res: ADF GLS 단위근 검정의 검정 통계량
    </br>  
>
> 참고: Ng and Perron (2001), Table I, pp. 1524
</br>


---

`adf_test_gls_detrending` 함수와 함께 사용되는 함수로는 단위근 검정 시 ADF 모형의 최적 시차를 추정하는 `mic` 함수가 있다. 이때, `mic`라는 함수는 다음과 구성되어 있다.

> *<p style="font-size:15px">`phat = mic(y, m, option=None)`</p>*
> *<p style="font-size:15px">수정된 정보기준을 통한 최적 시차 추정</p>*
> - **입력**:
</br>
    y: 추정하고자하는 변수, 단변수만 가능
    </br>
    m: 사용하고자하는 모형의 종류
    </br>
          'constant': 상수항만 존재하는 모형
        </br>
          'trend': 상수항과 선형 추세항이 존재하는 모형
        </br>
    ic: 수정된 정보기준의 종류
    </br>
          'maic': MAIC, penalty = 2
        </br>
          'mbic': MBIC, penalty = $\ln{(T-pmax-1)}$ (default)
        </br></br>
> - **출력**:
</br>
    phat: 추정된 최적 시차
    </br>

### (참고)
단위근 검정 시 사용하는 검정 통계량은 가정하고 있는 모형과 실제로 회귀를 통해 추정하는 모형의 형태에 따라 달라진다.

### (예시)
정리하면, 단위근 검정은 다음과 같은 절차를 통해 진행할 수 있다.
1. 수정된 정보기준 (MIC)를 사용한 ADF 모형의 최적 시차 추정
2. 추정된 최적 시차를 바탕으로 GLS 추세 제거를 한 ADF 검정  

이러한 결과에 대해 확인하기 위해 다음과 같이 실제 데이터를 활용하여 ADF-GLS 단위근 검정을 해보자. 우선 데이터의 경우 단위근이 존재하는 것으로 알려져있는 1976년 1분기부터 2023년 4분기까지의 미국 인플레이션 데이터를 사용한다.

In [2]:
df = pd.read_excel('data/python_data.xlsx', sheet_name='UScpi')
df = df.set_index('Date') # 라벨이 Date인 열을 인덱스(행 라벨)로 지정
df.index.name = None # df의 인덱스 이름 제거
df.index = pd.to_datetime(df.index) + pd.offsets.QuarterEnd(0) # 인덱스(행)를 날짜로 인식하도록 변환한다.
df.columns = ['US_inf'] # 열 이름을 US_inf로 지정
df

,US_inf
1976-03-31,6.148217
1976-06-30,5.841076
1976-09-30,5.439046
1976-12-31,5.056927
1977-03-31,5.735716
...,...
2022-12-31,6.869383
2023-03-31,5.609221
2023-06-30,3.972820
2023-09-30,3.503768


In [3]:
from bok_da.ts.test import adf_test_gls_detrending

In [10]:
# ADF test with GLS detrending
res = adf_test_gls_detrending(df, m='trend', print_res=True)

Modified BIC를 사용하여 추정된 ADF 모형의 AR 최적 시차는 8
 
상수항과 선형추세항이 함께 있는 모형을 사용한 ADF-GLS 검정통계량은 -2.2956
CV 1% =  -3.42  5% =  -2.91  10% =  -2.62
 
귀무가설 (H0): AR 단위근이 존재한다.
|test statistic| > |CV| : 귀무가설을 유의수준 x % 하에서 기각한다.
|test statistic| < |CV| : 귀무가설을 유의수준 x % 하에서 기각하지 못한다.
 
 ref: Ng and Perron (2001), <Table I>, pp. 1524


결과를 해석하면 수정된 베이지안 정보기준 (BIC)로 추정한 최적 시차 8을 가지는 ADF 모형을 GLS 추세 제거 이후 도출한 단위근 검정 통계량은 -2.2956이다. 이때, 회귀모형은 절편항과 선형 추세항이 모두 고려되었다. 이러한 검정 통계량을 입계값과 비교하면
$$|-2.2956| < |CV|$$
으로 AR 단위근이 존재한다는 귀무가설을 관용적인 10% 유의수준에서도 기각하지 못하는 것을 알 수 있다. 따라서 미국 인플레이션 데이터의 경우 단위근이 존재한다는 것을 알 수 있다.

참고로, `ic='maic'`로 설정하는 경우에도 귀무가설을 기각하지 못한다.

In [12]:
res = adf_test_gls_detrending(df, m='trend', ic='maic', print_res=True)

Modified AIC를 사용하여 추정된 ADF 모형의 AR 최적 시차는 12
 
상수항과 선형추세항이 함께 있는 모형을 사용한 ADF-GLS 검정통계량은 -2.451
CV 1% =  -3.42  5% =  -2.91  10% =  -2.62
 
귀무가설 (H0): AR 단위근이 존재한다.
|test statistic| > |CV| : 귀무가설을 유의수준 x % 하에서 기각한다.
|test statistic| < |CV| : 귀무가설을 유의수준 x % 하에서 기각하지 못한다.
 
 ref: Ng and Perron (2001), <Table I>, pp. 1524
